In [1]:
import math
import random

class value:
    def __init__(self, data, _children=(), _op="", label=""):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, value) else value(other)
        out = value(self.data + other.data, (self, other), "+")

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, value) else value(other)
        out = value(self.data * other.data, (self, other), "*")

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __sub__(self, other):
        return self + (-other)

    def __neg__(self):
        return self * -1

    def __truediv__(self, other):  # self / other
        return self * other**-1

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward

        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = value(t, (self,), "tanh")

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward

        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

# Generate synthetic data
def generate_data(n=100):
    data = []
    for _ in range(n):
        income = random.uniform(30, 150)
        roof_size = random.uniform(10, 50)
        sunlight = random.uniform(3, 8)
        label = 1 if (income > 70 and roof_size > 20 and sunlight > 5) else 0
        data.append(([value(income), value(roof_size), value(sunlight)], value(label)))
    return data

class Neuron:
    def __init__(self, nin):
        self.weights = [value(random.uniform(-1, 1)) for _ in range(nin)]
        self.bias = value(0.0)

    def __call__(self, x):
        act = sum((wi * xi for wi, xi in zip(self.weights, x)), self.bias)
        return act.tanh()

    def parameters(self):
        return self.weights + [self.bias]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

# Model architecture
model = Layer(3, 4)  # Input features: income, roof_size, sunlight
output_layer = Layer(4, 1)  # Single output (probability)

def forward(x):
    hidden = model(x)
    return output_layer(hidden)

def loss(y_pred, y_true):
    diff = y_pred - y_true
    return diff * diff  # Square the difference

# Training loop
data = generate_data(200)
learning_rate = 0.01

for epoch in range(100):
    total_loss = value(0.0)

    for x, y_true in data:
        # Forward pass
        y_pred = forward(x)
        # Compute loss
        total_loss = total_loss + loss(y_pred, y_true)

    # Backward pass
    total_loss.backward()

    # Update parameters
    for p in model.parameters() + output_layer.parameters():
        p.data -= learning_rate * p.grad
        p.grad = 0.0

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.data}")

# Test prediction
test_household = [value(80), value(30), value(6)]  # High income, large roof, sunny
pred = forward(test_household)
print(f"Solar panel adoption likelihood: {pred.data}")

Epoch 0, Loss: 117.54721923488945
Epoch 10, Loss: 128.908212007735
Epoch 20, Loss: 128.88784966987052
Epoch 30, Loss: 128.85598914178067
Epoch 40, Loss: 128.79920184120118
Epoch 50, Loss: 128.67037974947218
Epoch 60, Loss: 128.12134097514468
Epoch 70, Loss: 128.99999987173405
Epoch 80, Loss: 128.99999987173393
Epoch 90, Loss: 128.99999987173385
Solar panel adoption likelihood: 0.9999999995246056
